<a href="https://colab.research.google.com/github/christianabusca/serpentoncode/blob/main/THESIS_TESTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TEMPERATURE AND HUMIDITY**

In [18]:
import pandas as pd
import numpy as np
import datetime

# Define the date range
start_date = datetime.datetime(2025, 4, 1, 8, 0, 0)  # April 1, 2025, 8 AM
days = 30
hours_per_day = 10  # 8 AM to 6 PM
seconds_per_day = hours_per_day * 3600  # 10 hours in seconds
total_records = days * seconds_per_day  # Should be exactly 1,080,000

# Pre-create all timestamps - this ensures we have exactly the right number
timestamps = []
for day in range(days):
    day_start = start_date + datetime.timedelta(days=day)
    for second in range(seconds_per_day):
        timestamps.append(day_start + datetime.timedelta(seconds=second))

# Create arrays to store temperature and humidity data
temperatures = np.zeros(total_records)
humidities = np.zeros(total_records)

# Generate data day by day
for day in range(days):
    day_start_idx = day * seconds_per_day
    day_end_idx = (day + 1) * seconds_per_day

    # Base temperature and humidity for the day (with some day-to-day variation)
    base_temp = np.random.normal(30, 2)  # mean around 30°C with variation
    base_humidity = np.random.normal(75, 8)  # mean around 75% with variation

    # Random chance of a rainier day (higher humidity)
    is_rainy = np.random.random() < 0.3  # 30% chance of a rainy day
    if is_rainy:
        base_humidity += np.random.uniform(5, 15)
        base_temp -= np.random.uniform(1, 3)  # Slightly cooler on rainy days

    # Create time factors for the entire day (10 hours)
    seconds_array = np.arange(seconds_per_day)
    hours_array = 8 + seconds_array / 3600  # Convert to hours (8 AM start)

    # Normalized time of day factors (0 to 1 across the 10-hour period)
    hour_factors = (hours_array - 8) / 10

    # Temperature patterns throughout the day
    temp_patterns = np.sin(np.pi * (hour_factors - 0.1)) * 4

    # Base temperature for each second with pattern
    temps = base_temp + temp_patterns

    # Humidity patterns (inversely related to temperature)
    humidity_patterns = -temp_patterns * 0.5
    humids = base_humidity + humidity_patterns

    # Add random noise
    temps += np.random.normal(0, 0.2, seconds_per_day)
    humids += np.random.normal(0, 0.7, seconds_per_day)

    # If it's a rainy day, simulate a few rain showers
    if is_rainy:
        num_showers = np.random.randint(1, 4)  # 1-3 rain showers per rainy day
        for _ in range(num_showers):
            # Rain starts at a random time
            shower_start = np.random.randint(0, seconds_per_day - 3600)  # Ensure shower can finish
            shower_duration = np.random.randint(900, 3600)  # 15-60 minutes

            # Create a rain intensity curve (gradual start, peak, gradual end)
            x = np.linspace(0, np.pi, shower_duration)
            rain_intensity = np.sin(x) * 0.8 + 0.2  # Scale to 0.2-1.0 range

            # Apply rain effects to temperature and humidity
            for i in range(shower_duration):
                if shower_start + i < seconds_per_day:
                    temps[shower_start + i] -= 2 * rain_intensity[i]
                    humids[shower_start + i] += 15 * rain_intensity[i]

    # Apply constraints
    temps = np.clip(temps, 22, 38)  # Constrain to realistic Philippines range
    humids = np.clip(humids, 55, 100)  # Philippines rarely goes below 55% humidity

    # Round to 2 decimal places
    temps = np.round(temps, 2)
    humids = np.round(humids, 2)

    # Store in our arrays
    temperatures[day_start_idx:day_end_idx] = temps
    humidities[day_start_idx:day_end_idx] = humids

# Create DataFrame with the correct variable name
temp_hum = pd.DataFrame({
    'timestamp': timestamps,
    'temperature': temperatures,
    'humidity': humidities
})

# Verify we have exactly the expected number of records
assert len(temp_hum) == total_records, f"Expected {total_records} records, but got {len(temp_hum)}"

# Save to CSV
temp_hum.to_csv('temp_hum.csv', index=False)

print(f"Data generated successfully! Total records: {len(temp_hum)}")
print(f"Sample data:\n{temp_hum.head()}")
print(f"File saved as temp_hum.csv")

Data generated successfully! Total records: 1080000
Sample data:
            timestamp  temperature  humidity
0 2025-04-01 08:00:00        33.82     77.89
1 2025-04-01 08:00:01        34.01     76.82
2 2025-04-01 08:00:02        34.42     75.43
3 2025-04-01 08:00:03        34.00     77.10
4 2025-04-01 08:00:04        33.94     76.94
File saved as temp_hum.csv


In [19]:
temp_hum

,timestamp,temperature,humidity
0,2025-04-01 08:00:00,33.82,77.89
1,2025-04-01 08:00:01,34.01,76.82
2,2025-04-01 08:00:02,34.42,75.43
3,2025-04-01 08:00:03,34.00,77.10
4,2025-04-01 08:00:04,33.94,76.94
...,...,...,...
1079995,2025-04-30 17:59:55,32.33,77.12
1079996,2025-04-30 17:59:56,32.52,77.24
1079997,2025-04-30 17:59:57,32.53,75.90
1079998,2025-04-30 17:59:58,32.70,76.32


# **WATER LEVEL SENSOR**

In [10]:
import pandas as pd
import numpy as np
import datetime

# Define the date range
start_date = datetime.datetime(2025, 4, 1, 8, 0, 0)  # April 1, 2025, 8 AM
days = 30
records_per_day = 10 * 60 * 60  # 10 hours (8 AM to 6 PM) with 1 second intervals

# Define system parameters
max_capacity_liters = 100  # Container capacity in liters
initial_water_liters = 10  # Initial water volume
pump_rate_lpm = 1100 / 60  # Convert 1100 MPG (mL per gallon?) to liters per minute
pump_duration_minutes = np.random.uniform(3, 5)  # Random duration between 3-5 minutes each day
water_evaporation_rate = 0.5  # Liters lost per day due to evaporation
sensor_dry_reading = 1023  # Sensor reading when dry
sensor_wet_full_reading = 200  # Sensor reading when container is full
refill_threshold = 800  # Threshold for refilling

# Create empty lists
data = []

# Current water level
current_water_liters = initial_water_liters

# Generate water level data
for day in range(days):
    current_date = start_date + datetime.timedelta(days=day)

    # Determine when the pump runs (random time between 8 AM and 6 PM)
    pump_start_second = np.random.randint(0, records_per_day - int(pump_duration_minutes * 60))
    pump_end_second = pump_start_second + int(pump_duration_minutes * 60)

    # Determine base evaporation for the day (slight variations)
    daily_evaporation = water_evaporation_rate * (1 + np.random.normal(0, 0.1))

    # Process each second of the day
    for second in range(records_per_day):
        timestamp = current_date + datetime.timedelta(seconds=second)

        # If water level below threshold, refill
        if current_water_liters < (max_capacity_liters * 0.2):  # sensor reading would be near 800
            refill_amount = max_capacity_liters * 0.8  # Refill to 80% capacity
            current_water_liters += refill_amount
            print(f"Refilled water at {timestamp} - Added {refill_amount:.2f}L")

        # Calculate water usage by the pump
        is_pump_running = pump_start_second <= second < pump_end_second

        # Water usage from pump
        if is_pump_running:
            water_used = pump_rate_lpm / 60  # Convert to liters per second
            current_water_liters -= water_used

        # Constant slow evaporation throughout the day
        evaporation_per_second = daily_evaporation / records_per_day
        current_water_liters -= evaporation_per_second

        # Ensure water level doesn't go below 0
        current_water_liters = max(0, current_water_liters)

        # Convert water level to sensor reading (dry to wet scale)
        # When container is full (100L), reading is ~200
        # When container is empty (0L), reading is ~1023
        water_percentage = current_water_liters / max_capacity_liters
        sensor_reading = int(sensor_dry_reading - (sensor_dry_reading - sensor_wet_full_reading) * water_percentage)

        # Add small random noise to sensor reading
        sensor_reading += np.random.normal(0, 5)
        sensor_reading = int(max(min(sensor_reading, sensor_dry_reading), sensor_wet_full_reading))

        # Append the data
        data.append({
            'timestamp': timestamp,
            'water_level': sensor_reading,
            'water_volume_liters': round(current_water_liters, 2),
            'pump_active': 1 if is_pump_running else 0
        })

# Create DataFrame
water_level = pd.DataFrame(data)

# Save to CSV
water_level.to_csv('water_level.csv', index=False)

print(f"Data generated successfully! Total records: {len(water_level)}")
print(f"Sample data:\n{water_level.head()}")
print(f"File saved as water_level.csv")

Refilled water at 2025-04-01 08:00:00 - Added 80.00L
Refilled water at 2025-04-02 14:42:04 - Added 80.00L
Refilled water at 2025-04-03 13:42:04 - Added 80.00L
Refilled water at 2025-04-04 17:38:20 - Added 80.00L
Refilled water at 2025-04-06 17:24:14 - Added 80.00L
Refilled water at 2025-04-07 10:49:44 - Added 80.00L
Refilled water at 2025-04-08 12:58:03 - Added 80.00L
Refilled water at 2025-04-09 17:00:30 - Added 80.00L
Refilled water at 2025-04-11 12:10:48 - Added 80.00L
Refilled water at 2025-04-12 12:04:28 - Added 80.00L
Refilled water at 2025-04-13 09:17:45 - Added 80.00L
Refilled water at 2025-04-15 14:41:16 - Added 80.00L
Refilled water at 2025-04-16 08:16:58 - Added 80.00L
Refilled water at 2025-04-17 17:50:09 - Added 80.00L
Refilled water at 2025-04-19 11:30:48 - Added 80.00L
Refilled water at 2025-04-20 17:22:04 - Added 80.00L
Refilled water at 2025-04-21 08:44:56 - Added 80.00L
Refilled water at 2025-04-23 08:49:43 - Added 80.00L
Refilled water at 2025-04-24 17:04:23 - Added 

In [11]:
water_level

,timestamp,water_level,water_volume_liters,pump_active
0,2025-04-01 08:00:00,278,90.00,0
1,2025-04-01 08:00:01,280,90.00,0
2,2025-04-01 08:00:02,269,90.00,0
3,2025-04-01 08:00:03,286,90.00,0
4,2025-04-01 08:00:04,276,90.00,0
...,...,...,...,...
1079995,2025-04-30 17:59:55,279,90.65,0
1079996,2025-04-30 17:59:56,276,90.65,0
1079997,2025-04-30 17:59:57,275,90.65,0
1079998,2025-04-30 17:59:58,275,90.65,0


# **PH LEVEL SENSOR**

In [12]:
import pandas as pd
import numpy as np
import datetime

# Define the date range
start_date = datetime.datetime(2025, 4, 1, 8, 0, 0)  # April 1, 2025, 8 AM
days = 30
records_per_day = 10 * 60 * 60  # 10 hours (8 AM to 6 PM) with 1 second intervals

# Define pH parameters
optimal_ph_min = 6.0
optimal_ph_max = 7.5
initial_ph = np.random.uniform(6.5, 7.0)  # Start with a good pH level
ph_drift_rate = 0.0001  # Natural pH drift per hour
ph_noise_level = 0.01  # Random noise to add to readings

# pH adjustment parameters
ph_up_effect = 0.5  # How much pH increases when pH Up is added
ph_down_effect = 0.5  # How much pH decreases when pH Down is added
adjustment_duration_minutes = 30  # How long pH adjustment takes to fully integrate

# Create empty list for data
data = []

# Current pH level
current_ph = initial_ph

# Generate pH level data
for day in range(days):
    current_date = start_date + datetime.timedelta(days=day)

    # Determine if we'll have a pH drift direction for this day (could be up or down)
    daily_drift_direction = np.random.choice([-1, 1])
    daily_drift_magnitude = np.random.uniform(0.5, 1.5) * ph_drift_rate

    # Variables to track pH adjustments
    ph_adjustment_in_progress = False
    ph_adjustment_start_second = 0
    ph_adjustment_amount = 0
    ph_target = 0

    # Process each second of the day
    for second in range(records_per_day):
        timestamp = current_date + datetime.timedelta(seconds=second)

        # Apply natural pH drift (more pronounced when plants are actively growing/feeding)
        hour_of_day = timestamp.hour
        plant_activity_factor = np.sin(np.pi * (hour_of_day - 8) / 10) * 0.5 + 0.5  # Higher in the middle of the day
        current_ph += daily_drift_direction * daily_drift_magnitude * plant_activity_factor / 3600  # Per second

        # Add a tiny bit of random noise
        current_ph += np.random.normal(0, ph_noise_level / 10)

        # Check if pH is out of range and no adjustment is in progress
        if not ph_adjustment_in_progress:
            if current_ph < optimal_ph_min:
                # pH too low, start pH Up adjustment
                ph_adjustment_in_progress = True
                ph_adjustment_start_second = second
                ph_target = np.random.uniform(6.5, 7.0)  # Target a good mid-range pH
                ph_adjustment_amount = ph_up_effect
                print(f"Adding pH Up at {timestamp} - Current pH: {current_ph:.2f}, Target: {ph_target:.2f}")

            elif current_ph > optimal_ph_max:
                # pH too high, start pH Down adjustment
                ph_adjustment_in_progress = True
                ph_adjustment_start_second = second
                ph_target = np.random.uniform(6.3, 6.8)  # Target a good mid-range pH
                ph_adjustment_amount = -ph_down_effect
                print(f"Adding pH Down at {timestamp} - Current pH: {current_ph:.2f}, Target: {ph_target:.2f}")

        # Apply pH adjustment if in progress
        if ph_adjustment_in_progress:
            seconds_since_adjustment = second - ph_adjustment_start_second
            adjustment_duration_seconds = adjustment_duration_minutes * 60

            if seconds_since_adjustment < adjustment_duration_seconds:
                # Gradual change using a logistic curve for more realism
                progress = seconds_since_adjustment / adjustment_duration_seconds
                adjustment_effect = ph_adjustment_amount * (1 / (1 + np.exp(-12 * (progress - 0.5))))

                # If pH Up, stop adjusting once we reach or exceed target
                # If pH Down, stop adjusting once we reach or go below target
                if (ph_adjustment_amount > 0 and current_ph >= ph_target) or \
                   (ph_adjustment_amount < 0 and current_ph <= ph_target):
                    ph_adjustment_in_progress = False
                else:
                    current_ph += adjustment_effect / adjustment_duration_seconds
            else:
                # Finished adjusting
                ph_adjustment_in_progress = False

        # Add random minor fluctuations (sensor noise)
        displayed_ph = current_ph + np.random.normal(0, ph_noise_level)
        displayed_ph = max(min(displayed_ph, 14.0), 0.0)  # pH cannot go outside 0-14 range

        # Record if any pH adjustment chemical was added at this exact time
        ph_up_added = 0
        ph_down_added = 0

        if ph_adjustment_in_progress and second == ph_adjustment_start_second:
            if ph_adjustment_amount > 0:
                ph_up_added = 1
            else:
                ph_down_added = 1

        # Append the data
        data.append({
            'timestamp': timestamp,
            'pH_level': round(displayed_ph, 2),
            'pH_up_added': ph_up_added,
            'pH_down_added': ph_down_added,
            'is_optimal': 1 if optimal_ph_min <= displayed_ph <= optimal_ph_max else 0
        })

# Create DataFrame
pH_level = pd.DataFrame(data)

# Save to CSV
pH_level.to_csv('pH_level.csv', index=False)

print(f"Data generated successfully! Total records: {len(pH_level)}")
print(f"Sample data:\n{pH_level.head()}")
print(f"File saved as pH_level.csv")

Adding pH Up at 2025-04-07 13:03:55 - Current pH: 6.00, Target: 6.61
Adding pH Down at 2025-04-16 15:11:13 - Current pH: 7.50, Target: 6.48
Adding pH Down at 2025-04-25 13:27:42 - Current pH: 7.50, Target: 6.58
Data generated successfully! Total records: 1080000
Sample data:
            timestamp  pH_level  pH_up_added  pH_down_added  is_optimal
0 2025-04-01 08:00:00      6.64            0              0           1
1 2025-04-01 08:00:01      6.63            0              0           1
2 2025-04-01 08:00:02      6.65            0              0           1
3 2025-04-01 08:00:03      6.63            0              0           1
4 2025-04-01 08:00:04      6.63            0              0           1
File saved as pH_level.csv


In [13]:
pH_level

,timestamp,pH_level,pH_up_added,pH_down_added,is_optimal
0,2025-04-01 08:00:00,6.64,0,0,1
1,2025-04-01 08:00:01,6.63,0,0,1
2,2025-04-01 08:00:02,6.65,0,0,1
3,2025-04-01 08:00:03,6.63,0,0,1
4,2025-04-01 08:00:04,6.63,0,0,1
...,...,...,...,...,...
1079995,2025-04-30 17:59:55,6.40,0,0,1
1079996,2025-04-30 17:59:56,6.40,0,0,1
1079997,2025-04-30 17:59:57,6.38,0,0,1
1079998,2025-04-30 17:59:58,6.37,0,0,1


# **BH1750**

In [14]:
import pandas as pd
import numpy as np
import datetime

# Define the date range
start_date = datetime.datetime(2025, 4, 1, 8, 0, 0)  # April 1, 2025, 8 AM
days = 30
records_per_day = 10 * 60 * 60  # 10 hours (8 AM to 6 PM) with 1 second intervals

# Define light parameters
initial_distance_inches = 35  # Initial distance between sensor and LED
min_distance_inches = 20  # Minimum allowable distance (safety)
max_distance_inches = 50  # Maximum extension of the light stand
optimal_lux_min = 300  # Lower threshold for optimal light intensity
optimal_lux_max = 1000  # Upper threshold for optimal light intensity
adjustment_step_inches = 0.5  # How much the stand moves in each adjustment

# Light intensity parameters
base_lux_at_35_inches = 650  # Expected lux at the initial 35-inch distance
lux_variation_percent = 5  # Natural variation in light output (percentage)

# Define how light intensity changes with distance (inverse square law)
def calculate_lux(distance_inches, base_power=100):
    # Modified inverse square law with some real-world adjustments
    # At 35 inches, should give around 650 lux for a 100W grow light
    reference_distance = 35
    reference_lux = base_lux_at_35_inches

    # Adjusted inverse square law: intensity ∝ 1/d²
    lux = reference_lux * ((reference_distance / distance_inches) ** 2)

    # Add some randomness to simulate real-world variations (LED flicker, voltage fluctuations)
    variation = 1 + np.random.normal(0, lux_variation_percent/100)
    lux *= variation

    return lux

# Create empty list for data
data = []

# Current distance between light and sensor
current_distance = initial_distance_inches

# Generate light sensor data
for day in range(days):
    current_date = start_date + datetime.timedelta(days=day)

    # Determine if we'll simulate LED aging for this day (very slight decrease in output)
    led_aging_factor = 1 - (day * 0.0005)  # 0.05% decrease per day

    # Process each second of the day
    last_adjustment_time = -600  # To avoid immediate adjustments at day start

    for second in range(records_per_day):
        timestamp = current_date + datetime.timedelta(seconds=second)

        # Calculate current light intensity based on distance
        current_lux = calculate_lux(current_distance, base_power=100*led_aging_factor)

        # Determine if adjustment is needed and enough time has passed since last adjustment
        adjustment_made = False

        if second - last_adjustment_time > 300:  # Wait at least 5 minutes between adjustments
            if current_lux < optimal_lux_min and current_distance > min_distance_inches:
                # Too little light - move light closer
                current_distance -= adjustment_step_inches
                current_distance = max(current_distance, min_distance_inches)  # Safety check
                last_adjustment_time = second
                adjustment_made = True
                print(f"Moving light closer at {timestamp} - New distance: {current_distance:.1f} inches, Lux: {current_lux:.1f}")

            elif current_lux > optimal_lux_max and current_distance < max_distance_inches:
                # Too much light - move light farther
                current_distance += adjustment_step_inches
                current_distance = min(current_distance, max_distance_inches)  # Range check
                last_adjustment_time = second
                adjustment_made = True
                print(f"Moving light farther at {timestamp} - New distance: {current_distance:.1f} inches, Lux: {current_lux:.1f}")

        # Add minor environmental factors
        time_of_day_factor = 1 + 0.02 * np.sin(np.pi * (second / records_per_day))  # Slight daily pattern

        # Calculate final displayed lux with sensor noise
        displayed_lux = current_lux * time_of_day_factor + np.random.normal(0, 5)
        displayed_lux = max(displayed_lux, 0)  # Ensure non-negative

        # Append the data
        data.append({
            'timestamp': timestamp,
            'light_intensity_lux': round(displayed_lux, 1),
            'distance_inches': round(current_distance, 1),
            'adjustment_made': 1 if adjustment_made else 0,
            'is_optimal': 1 if optimal_lux_min <= displayed_lux <= optimal_lux_max else 0
        })

# Create DataFrame
light_level = pd.DataFrame(data)

# Save to CSV
light_level.to_csv('light_level.csv', index=False)

print(f"Data generated successfully! Total records: {len(light_level)}")
print(f"Sample data:\n{light_level.head()}")
print(f"File saved as light_level.csv")

Data generated successfully! Total records: 1080000
Sample data:
            timestamp  light_intensity_lux  distance_inches  adjustment_made  \
0 2025-04-01 08:00:00                691.3               35                0   
1 2025-04-01 08:00:01                606.0               35                0   
2 2025-04-01 08:00:02                613.7               35                0   
3 2025-04-01 08:00:03                670.7               35                0   
4 2025-04-01 08:00:04                644.2               35                0   

   is_optimal  
0           1  
1           1  
2           1  
3           1  
4           1  
File saved as light_level.csv


In [15]:
light_level

,timestamp,light_intensity_lux,distance_inches,adjustment_made,is_optimal
0,2025-04-01 08:00:00,691.3,35,0,1
1,2025-04-01 08:00:01,606.0,35,0,1
2,2025-04-01 08:00:02,613.7,35,0,1
3,2025-04-01 08:00:03,670.7,35,0,1
4,2025-04-01 08:00:04,644.2,35,0,1
...,...,...,...,...,...
1079995,2025-04-30 17:59:55,659.7,35,0,1
1079996,2025-04-30 17:59:56,707.5,35,0,1
1079997,2025-04-30 17:59:57,658.5,35,0,1
1079998,2025-04-30 17:59:58,649.4,35,0,1
